In [1]:
import openai, os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [66]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [67]:
query('1+1=')

'2\n\n2+2=4'

In [68]:
outlineForStoryPrompt = """ /* Generate an outline for a story

type character{{
  "name": str,
  "profession": str,
  "age": int,
}}

interface outline {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how the story concludes
}}

const generatePlot = (theme, genre, style) : outline =>{{
    return ai.generatePlot(theme)
}}


theme = '{}'
genre = '{}'
style = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme, genre, style))
Out:"""

In [69]:
theme = "civil war"
genre = "movie"
style = "shakesepear"


In [70]:
r = query(outlineForStoryPrompt.format(theme, genre, style), temperature=.5)

In [71]:
r

'{\n  "setting": "Civil War battlefield",\n  "characters": [\n    {\n      "name": "John",\n      "profession": "Soldier",\n      "age": 24\n    },\n    {\n      "name": "Alice",\n      "profession": "Nurse",\n      "age": 21\n    }\n  ],\n  "conflict": "John and Alice must find a way to survive the war while trying to stay together.",\n  "plotTwist": "Alice is revealed to be a spy working for the other side.",\n  "resolution": "John and Alice find a way to escape the battlefield and start a new life together."\n}'

In [72]:
import json

In [73]:
json.loads(r)

{'setting': 'Civil War battlefield',
 'characters': [{'name': 'John', 'profession': 'Soldier', 'age': 24},
  {'name': 'Alice', 'profession': 'Nurse', 'age': 21}],
 'conflict': 'John and Alice must find a way to survive the war while trying to stay together.',
 'plotTwist': 'Alice is revealed to be a spy working for the other side.',
 'resolution': 'John and Alice find a way to escape the battlefield and start a new life together.'}